In [1]:
import torch

root_path = "/home/yigit/projects/mbcnp/outputs/experimental/56D/"
run_id, model_folder = '1702213155_cw', '/saved_models/'
models_path = f'{root_path}{run_id}{model_folder}'

cnep_path = f'{models_path}wta_on_synth.pt'

y = torch.load(f'{root_path}{run_id}/y.pt').cpu()
num_samples, t_steps, dy = y.shape
dx = 1 

print(num_samples, t_steps, dy)

# x = torch.linspace(0, 1, t_steps).repeat(num_samples, 1).view(num_samples, -1, dx)

16 122 56


In [2]:
from models.wta_cnp import WTA_CNP

batch_size = 1
n_max_obs, n_max_tar = 5, 5
cnep = WTA_CNP(dx, dy, n_max_obs, n_max_tar, [2048, 1536, 1024], num_decoders=2, decoder_hidden_dims=[1024, 1024, 1024], batch_size=batch_size)

cnep.load_state_dict(torch.load(cnep_path))

cnep.eval()

WTA_CNP(
  (encoder): Sequential(
    (0): Linear(in_features=57, out_features=2048, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2048, out_features=1536, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1536, out_features=1024, bias=True)
  )
  (decoders): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=1025, out_features=1024, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1024, out_features=1024, bias=True)
      (3): ReLU()
      (4): Linear(in_features=1024, out_features=112, bias=True)
    )
  )
  (gate): Sequential(
    (0): Linear(in_features=1024, out_features=2, bias=True)
    (1): Softmax(dim=-1)
  )
)

In [3]:
from dm_control.locomotion.mocap import cmu_mocap_data
from dm_control.locomotion.mocap import loader
from dm_control.locomotion.tasks.reference_pose import tracking
from dm_control.locomotion.tasks.reference_pose import utils
from dm_control.locomotion.walkers import initializers
import numpy as np
import tree
import mujoco

class StandInitializer(initializers.WalkerInitializer):
    def __init__(self):
        ref_path = cmu_mocap_data.get_path_for_cmu(version='2020')
        mocap_loader = loader.HDF5TrajectoryLoader(ref_path)
        trajectory = mocap_loader.get_trajectory('CMU_049_06')
        clip_reference_features = trajectory.as_dict()
        clip_reference_features = tracking._strip_reference_prefix(clip_reference_features, 'walker/')
        self._stand_features = tree.map_structure(lambda x: x[0], clip_reference_features)

    def initialize_pose(self, physics, walker, random_state):
        del random_state
        utils.set_walker_from_features(physics, walker, self._stand_features)
        mujoco.mj_kinematics(physics.model.ptr, physics.data.ptr)

In [8]:
import time
from dm_control import viewer
from dm_control import composer
from dm_control.locomotion import arenas
from dm_control.locomotion.tasks import go_to_target
from dm_control.locomotion.walkers import cmu_humanoid



from typing import Any, Callable, Dict, Optional, Text, Tuple
from dm_control import composer
from dm_control.locomotion.mocap import cmu_mocap_data
from dm_control.locomotion.mocap import loader
from dm_control.locomotion.tasks.reference_pose import tracking
from dm_control.locomotion.tasks.reference_pose import utils
from dm_control.locomotion.walkers import cmu_humanoid
from dm_control.locomotion.walkers import initializers

class CartwheelInitializer(initializers.WalkerInitializer):
    def __init__(self):
        ref_path = cmu_mocap_data.get_path_for_cmu(version='2020')
        mocap_loader = loader.HDF5TrajectoryLoader(ref_path)
        trajectory = mocap_loader.get_trajectory('CMU_049_06')
        self._trajectory = trajectory
        clip_reference_features = trajectory.as_dict()
        clip_reference_features = tracking._strip_reference_prefix(clip_reference_features, 'walker/')
        self._cw_features = tree.map_structure(lambda x: x[0], clip_reference_features)

    def initialize_pose(self, physics, walker, random_state):
        del random_state
        utils.set_walker_from_features(physics, walker, self._cw_features)
        mujoco.mj_kinematics(physics.model.ptr, physics.data.ptr)


cnep_o = torch.zeros((batch_size, 1, dx+dy))
cnep_t = torch.zeros((batch_size, 1, dx))
tind = 10

def prepare_obs(i):
    global cnep_o, cnep_t, tind
    cnep_t[0, 0, 0] = i+1
    
    cnep_o[0, 0, 0] = i
    cnep_o[0, 0, dx:] = y[tind, 0, :]

    return cnep_o, cnep_t

initializer = CartwheelInitializer()
walker = cmu_humanoid.CMUHumanoidPositionControlledV2020(initializer=initializer)
arena = arenas.Floor()

task = go_to_target.GoToTarget(walker=walker, arena=arena, physics_timestep=0.005, control_timestep=0.03)
env = composer.Environment(task=task, random_state=0)
# print(env.control_timestep())

initializer.initialize_pose(env.physics, walker, 0)

ind = -1
# # Viewer for visualization
# viewer.launch(env)
# # print(te[3])

# for i in range(t_steps):
#     o, t = prepare_obs(i)
#     preds, _ = cnep(o, t)
#     a = preds
#     s = env.step(a)
#     # print(s)
#     # print(s[3].shape)

def tst(ts):
    global ind
    ind += 1
    
    o, t = prepare_obs(ind)
    preds, gate = cnep(o, t)

    inst_a = preds[torch.argmax(gate.squeeze(1), dim=-1), 0, 0, :dy].cpu().detach().numpy().squeeze()
    return inst_a
    return y[tind, ind].numpy()

# Viewer for visualization
viewer.launch(env, policy=tst)



env.close()